# Bascis

- CRS information **connects** data to the Earth's surface using a mathematical model.
- **threee Components**: A model of the shape of the earth (datum, e.g., WGS84); A mathematical transformation of the angular measurements on a round earth to a flat surface (e.g., UTM projections); other Parameters.
- In general CRS can be divided into projected coordinate reference systems (also called Cartesian or rectangular coordinate reference systems) and geographic coordinate reference systems

# Common systems for describing CRSs include EPSG, PROJ, and OGC WKT.

## A PROJ4 string includes the following information:

- proj=: the projection of the data
- zone=: the zone of the data (this is specific to the UTM projection)
- datum=: the datum use
- units=: the units for the coordinates of the data
- ellps=: the ellipsoid (how the earth's roundness is calculated) for the data

examples:

+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0

+proj=utm +zone=11 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0

## EPSG

The EPSG Geodetic Parameter Dataset is a dataset of coordinate reference systems and coordinate transformations.

It is represented by a 4- to 5-digit code (e.g. EPSG:4326 = WGS84). The codes can be global, regional, national or local in application.

# Related Code, in particular for convertion and EPSG calculation

We can use gdal to get the CRS info. of a file, e.g., by:
 ```
$ gdalsrsinfo EPSG:4326

PROJ.4 : +proj=longlat +datum=WGS84 +no_defs
$ gdalsrsinfo -o proj4 *.tif
 ```

Transformation from one projection to another is common and easy. For instance, you have a point in WGS84 Latitude/Longitude  (i.e., "EPSG:4326"), you can convert it to **any projections**, e.g.,UTM Zones (North): "EPSG:32633", and UTM Zones (South): "EPSG:32733", in theory. The question is:

- **which one** do you need? 
- Further, how do you know which one do you need?

## 1 with geopandas

In [39]:
import pyproj

#note that any recent version of geopandas requires at least pyproj>=2.0.0.

crs = pyproj.CRS("+proj=laea +lat_0=45 +lon_0=-100 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs")
crs.to_epsg()

2163

In [40]:
import geopandas as gpd
from shapely.geometry import Point

lat = 45
lon = -100
d = {'geometry': [Point(), Point(lon,lat)]}
print(d)
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")#WGS84 Latitude/Longitude: "EPSG:4326"
print(gdf, gdf.crs)

utm_zone = int(math.floor((lon + 180) / 6) + 1 )
print("Zone is",utm_zone)

# add this new zone value to the following string format
# so you see it is important to know these different (parallel) descriptions of CRS
utm_crs = '+proj=utm +zone={} +ellps=WGS84 +datum=WGS84 +units=m +no_defs'.format(utm_zone)              
# use geopandas to trigger a reprojection of the geodataframe
reproj_gdf = gdf.to_crs(utm_crs)
print(reproj_gdf)

crs = pyproj.CRS(utm_crs)
print(crs.to_epsg())

#with epsg code, we can again convert the point
gdf.to_crs(crs.to_epsg())

{'geometry': [<shapely.geometry.point.Point object at 0x00000256A57A6FD0>, <shapely.geometry.point.Point object at 0x00000256A57A6358>]}
                   geometry
0  GEOMETRYCOLLECTION EMPTY
1           POINT (-100 45) EPSG:4326
Zone is 14
                                     geometry
0                    GEOMETRYCOLLECTION EMPTY
1  POINT (421184.697083289 4983436.768349296)
32614


,geometry
0,GEOMETRYCOLLECTION EMPTY
1,POINT (421184.697083289 4983436.768349296)


## 2 with math in python to find the EPSG code of a UTM projection

In [41]:
#with math in python
import numpy as np

lat = 45
lon = -100

utmZone = np.round((183+lon)/6,0) 
print("Zone is",utmZone)
EPSG = 32700-np.round((45+lat)/90,0)*100+np.round((183+lon)/6,0)
EPSG = int(EPSG)
print("EPSG is",EPSG)

Zone is 14.0
EPSG is 32614


In [42]:
import math

lat = 45
lon = -100

utmZone = str((math.floor((lon + 180) / 6) % 60) + 1)
if len(utmZone) == 1:
    utmZone = '0' + utmZone
if lat >= 0:
    epsg_code = '326' + utmZone  # lat>0: N;
else:
    epsg_code = '327' + utmZone
    
epsg_code

'32614'

## 3 with math in gee

In [43]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

#create a Point in gee
lat = 45
lon = -100
Point = ee.Geometry.Point([lon,lat])
print(Point.getInfo())

# Create a Feature from the Geometry.
pointFeature = ee.Feature(Point)

coords = Point.coordinates()

lon = ee.Number(coords.get(0))
lat = ee.Number(coords.get(1))
epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()
print(epsg.getInfo())

#note that gee has its unique syntax
pointFeature_ = pointFeature.set("EPSG",ee.String("EPSG:").cat(ee.String(str(epsg.getInfo()))))
print(pointFeature_.getInfo())

print(Point.transform(ee.String("EPSG:").cat(ee.String(str(epsg.getInfo())))).getInfo())

{'type': 'Point', 'coordinates': [-100, 45]}
32614
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-100, 45]}, 'properties': {'EPSG': 'EPSG:32614'}}
{'geodesic': False, 'crs': {'type': 'name', 'properties': {'name': 'EPSG:32614'}}, 'type': 'Point', 'coordinates': [421184.6970832809, 4983436.768350039]}


# Ref

- [Video](https://www.youtube.com/embed/KUF_Ckv8HbE) highlighting how map projections can make continents seems proportionally larger or smaller than they actually are.
- [Documentation for QGIS](https://docs.qgis.org/3.10/en/docs/user_manual/working_with_projections/working_with_projections.html)
- [Docs of GeoPandas](https://geopandas.org/projections.html#what-is-the-best-format-to-store-the-crs-information)